In [1]:
import pandas as pd
import altair as alt

In [2]:
df = pd.read_csv('~/figures_data/excess-mortality-raw-death-count.csv')
df['Entity'].unique()

array(['Albania', 'Andorra', 'Armenia', 'Aruba', 'Australia', 'Austria',
       'Azerbaijan', 'Belarus', 'Belgium', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Canada', 'Chile',
       'Colombia', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia',
       'Denmark', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Finland',
       'France', 'French Guiana', 'French Polynesia', 'Georgia',
       'Germany', 'Gibraltar', 'Greece', 'Greenland', 'Guadeloupe',
       'Guatemala', 'Hong Kong', 'Hungary', 'Iceland', 'Ireland',
       'Israel', 'Italy', 'Jamaica', 'Japan', 'Kazakhstan', 'Kyrgyzstan',
       'Latvia', 'Lebanon', 'Liechtenstein', 'Lithuania', 'Luxembourg',
       'Macao', 'Malaysia', 'Malta', 'Martinique', 'Mauritius', 'Mayotte',
       'Mexico', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro',
       'Netherlands', 'New Zealand', 'Nicaragua', 'North Macedonia',
       'Norway', 'Oman', 'Panama', 'Paraguay', 'Peru', 'Philippines',
       'Poland', 'Portugal',

In [3]:
df = df[df['Entity'].isin(['United Kingdom','United States','Spain','Germany','Italy','Taiwan'])]
df['cumulative_deaths_2020_all_ages'] = df.groupby('Entity')['deaths_2020_all_ages'].transform(pd.Series.cumsum)
df['cumulative_average_deaths_2015_2019_all_ages'] = df.groupby('Entity')['average_deaths_2015_2019_all_ages'].transform(pd.Series.cumsum)
df['cumulative_excess_death_2020'] = df['cumulative_deaths_2020_all_ages'] - df['cumulative_average_deaths_2015_2019_all_ages']
df['2020 cumulative excess death per million'] = 1e6 * df['cumulative_excess_death_2020'] / df['Population']

In [4]:
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Day'], empty='none')

# The basic line
line = alt.Chart(df).mark_line(interpolate='basis').encode(
    x='Day',
    y='2020 cumulative excess death per million',
    color='Entity'
)


# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(df).mark_point().encode(
    x='Day',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, '2020 cumulative excess death per million', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(df).mark_rule(color='gray').encode(
    x='Day',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
figure = alt.layer(
    line, selectors, points, rules, text
).properties(
    width=600, height=300
)
figure

alt.LayerChart(...)

In [5]:
#figure.save('~/figures_data/excess_mortality_2020.png', scale_factor=2.0)

KeyboardInterrupt: 

In [6]:
from altair_saver import save

In [7]:
save(figure, '~/figures_data/excess_mortality_2020.png')

KeyboardInterrupt: 